<a target="_blank" href="https://colab.research.google.com/github/JuanGoezD/prueba-seleccion-IO1">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 3. Enunciado Problema
Resolver el siguiente ejercicio en GoogleColab con la librería cvxpy utilizando al menos dos solvers diferentes y comparando las respuestas generadas por cada uno de ellos (el cálculo de las diferencias debe ser realizado de manera automática por el código).

Minimizar 3*x + 8*y

sujeto a:

x + y >= 50

x - y <= 20

x >= 0

y >= 0

In [4]:
# Importar las librerías que se van a utilizar
import cvxpy as cvx
# El siguiente  módulo se va a usar para la impresión de resultados
from tabulate import tabulate

# Se crean las variables
# Además se utiliza la restriccion de no negatividad de las variables
x = cvx.Variable(nonneg = True)
y = cvx.Variable(nonneg = True)

# Se define la función objetivo
func_obj = cvx.Minimize(3*x + 8*y)

# Establecemos las restricciones
restricciones = [
x + y >= 50,
x - y <= 20
]

# Configuro dos problemas, uno con el solver OSQP y otro con el solver SCS

# Soluciones con el solver OSQP
prob_OSQP = cvx.Problem(func_obj, restricciones)
prob_OSQP.solve(solver=cvx.OSQP) # Soluciono el problema con OSQP
valorx_OSQP = x.value # Valores optimos de x con OSQP
valory_OSQP = y.value # Valores optimos de y con OSQP

# Soluciones con el solver SCS
prob_SCS = cvx.Problem(func_obj, restricciones)
prob_SCS.solve(solver=cvx.SCS) # Soluciono el problema con SCS
valorx_SCS = x.value # Valores optimos de x con SCS
valory_SCS = y.value # Valores optimos de y con SCS

# Se crea una lista con los encabezados de la tabla
encabezado = ["Solver","Estado de la solución", "Valor óptimo de la función objetivo",
        "Valor óptimo x", "Valor óptimo de y"]

# Lista con los resultados de la solución del solver y valores de las
# Variables de decisión
datos = [["OSQP", prob_OSQP.status, prob_OSQP.value, valorx_OSQP, valory_OSQP ],
        ["SCS", prob_SCS.status, prob_SCS.value, valorx_SCS, valory_SCS]]

print("Los resultados de ambos solvers son: \n")
# Se imprime la tabla con los resultados, dándole un formato de grid y
# Y utilizando todos los números decimales
print(tabulate(datos, headers=encabezado, tablefmt = "grid",
               numalign="rigth", floatfmt= ".14f" ))

print("\nLas diferencias absolutas entre los solvers son: ")

# Lista con las diferencias absolutas entre los resultados de las
# Variables de decisión y función objetivo
datos_dif = [
     ["Diferencia en el valor x", abs(valorx_OSQP - valorx_SCS)],
     ["Diferencia en el valor y", abs(valory_OSQP - valory_SCS)],
    ["Diferencia en la funcion objetivo", abs(prob_OSQP.value - prob_SCS.value)]
             ]

# Se imprime la tabla con los resultados de las diferencias
print(tabulate(datos_dif, tablefmt="grid", floatfmt=".14f"))

Los resultados de ambos solvers son: 

+----------+-------------------------+---------------------------------------+-------------------+---------------------+
| Solver   | Estado de la solución   |   Valor óptimo de la función objetivo |    Valor óptimo x |   Valor óptimo de y |
+==========+=========================+=======================================+===================+=====================+
| OSQP     | optimal                 | 225.00000000000000                    | 35.00000000000000 | 15.00000000000000   |
+----------+-------------------------+---------------------------------------+-------------------+---------------------+
| SCS      | optimal                 | 225.00005534246503                    | 35.00001549417795 | 15.00000110749139   |
+----------+-------------------------+---------------------------------------+-------------------+---------------------+

Las diferencias absolutas entre los solvers son: 
+-----------------------------------+------------------+
| Dife

# 4. Enunciado problema
Resolver el siguiente ejercicio en GoogleColab con la librería cvxpy utilizando al menos dos solvers diferentes y comparando las respuestas generadas por cada uno de ellos (el cálculo de las diferencias debe ser realizado de manera automática por el código).

Minimizar 3*x + 8*y

sujeto a:

x + y >= 50

x - y <= 20

x >= 0

y >= 0

Debido a cómo trabaja Scipy se trabajarán con las siguientes restricciones

-x -y <= -50

x - y <= 20

Y las de no negatividad

## Consideraciones importantes de la libreria de Scipy
La librería de Scipy no es tan permisiva ni tan fácil de escribir matemáticamente como cvxpy. Por tanto hay 3 cosas a tener en cuenta:



*   Tenemos que trabajar con los coeficientes que acompañan tanto a la función objetivo como a las restricciones. La posición de la lista en la que esté el coeficiente representará la variable de decisión.
*   Todas las desigualdades deben ser  $\leq$ Si están de la forma $\geq$ deben ser multiplicadas por $-1$.

* El módulo que se va a usar sólo resuelve por minimización, es decir, para maximizar una función objetivo tendríamos que resolver el problema dual. Pero esto es tan sencillo como multiplicar la función objetivo por $-1$ y trabajar con esos coeficientes.

Debido a que el ejercicio nos pide minimizar, lo ultimo no se realizará, con esto en cuenta podemos desarrollarlo utilizando el módulo de linprog de Scipy.

In [5]:
# Importamos los modulos a utilizar
from scipy.optimize import linprog
from tabulate import tabulate

# Coeficientes de la función objetivo
c = [3, 8]

# Se crea una lista de lista que contendrá los coeficientes de
# las restricciones del problema
A_ub = [[-1, -1],
        [1, -1]]

# Se crea otra lista con los valores derechos de la desigualdad
b_ub = [-50, 20]

# Por defecto, Scipy utiliza las restricciones de no negatividad
# Pero para dar más claridad se hará explicitas en este código
# None sirve como infinito (también se puede usar np.inf si se desea)
# Como par de valores, None a la izquierda es -inf, y None a la derecha es +inf
bounds = [(0, None),
          (0, None)]

# Se definen los métodos por los cuales se resolverá
method_interior_point = "interior-point"
method_high    = "highs"

# Resolvemos el problema con el método del punto interior
problem_interior_point = linprog(c, A_ub = A_ub, b_ub = b_ub,
                          bounds = bounds, method = method_interior_point)

# Resolvemos el problema con el método highs
# Por defecto scipy utiliza el método highs, pero para claridad en el código
# Se hace expreso su uso
problem_high = linprog(c, A_ub = A_ub, b_ub = b_ub,
                          bounds = bounds, method = method_high)

# Se crea una lista con los encabezados de la tabla
encabezado = ["Solver","Estado de la solución", "Valor óptimo de la función objetivo",
        "Valor óptimo x", "Valor óptimo de y"]

# Lista con los resultados de la solución del solver
# Y valores de las variables de decisión
datos = [["Interior point", problem_interior_point.status,
         problem_interior_point.fun,
         problem_interior_point.x[0], problem_interior_point.x[1] ],
        ["SCS", problem_high.status,
         problem_high.fun,
         problem_high.x[0], problem_high.x[1]]]

print("Los resultados de ambos solvers son: \n")
# Se imprime la tabla con los resultados, dándole un formato de grid y
# Y utilizando todos los números decimales, Si el estado es 0 significa
# Que la optimización ocurrió adecuadamente

print(tabulate(datos, headers=encabezado, tablefmt = "grid",
               numalign="rigth", floatfmt= ".14f" ))

print("\nLas diferencias absolutas entre los solvers son: ")

# Lista con las diferencias absolutas entre los resultados de las
# Variables de decisión y función objetivo
datos_dif = [
["Diferencia en x", abs(problem_interior_point.x[0] - problem_high.x[0])],
["Diferencia en y", abs(problem_interior_point.x[1] - problem_high.x[1])],
["Diferencia en solución", abs(problem_interior_point.fun - problem_high.fun)]
             ]
# Se imprime la tabla con los resultados de las diferencias
print(tabulate(datos_dif, tablefmt="grid", floatfmt=".14f"))

Los resultados de ambos solvers son: 

+----------------+-------------------------+---------------------------------------+-------------------+---------------------+
| Solver         |   Estado de la solución |   Valor óptimo de la función objetivo |    Valor óptimo x |   Valor óptimo de y |
+================+=========================+=======================================+===================+=====================+
| Interior point | 0                       | 224.99999484531867                    | 34.99999919998080 | 14.99999965567204   |
+----------------+-------------------------+---------------------------------------+-------------------+---------------------+
| SCS            | 0                       | 225.00000000000000                    | 35.00000000000000 | 15.00000000000000   |
+----------------+-------------------------+---------------------------------------+-------------------+---------------------+

Las diferencias absolutas entre los solvers son: 
+--------------------

<ipython-input-5-09ed12490105>:28: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  problem_interior_point = linprog(c, A_ub = A_ub, b_ub = b_ub,
